# Real-case gravity inversion

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile

from discretize import TensorMesh
from discretize.utils import active_from_xyz
from SimPEG.potential_fields import magnetics
from SimPEG.utils import plot2Ddata, model_builder
from SimPEG import (
    maps,
    data,
    inverse_problem,
    data_misfit,
    regularization,
    optimization,
    directives,
    inversion,
    utils,
)
topo_filename = "t-mag.txt"
data_filename = "Gravity observation.txt"
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile

from discretize import TensorMesh
from discretize.utils import active_from_xyz
from SimPEG.utils import plot2Ddata, model_builder
from SimPEG.potential_fields import gravity
from SimPEG import (
    maps,
    data,
    data_misfit,
    inverse_problem,
    regularization,
    optimization,
    directives,
    inversion,
    utils,
)

# sphinx_gallery_thumbnail_number = 3
# Load topography
xyz_topo = np.loadtxt(str(topo_filename))

# Load field data
dobs = np.loadtxt(str(data_filename))

# Define receiver locations and observed data
receiver_locations = dobs[:, 0:3]
dobs = dobs[:, -1]

# Plot
mpl.rcParams.update({"font.size": 12})
fig = plt.figure(figsize=(9, 7))

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.85])
plot2Ddata(receiver_locations, dobs, ax=ax1,  ncontour=24, contourOpts={"cmap": "RdBu_r"},)
#ax1.set_title("Gravity Observation",size = 16.5)
ax1.set_xlabel("Easting (m)",size = 15.5,labelpad=25)
ax1.set_ylabel("Northing (m)",size = 15.5,labelpad=25)
ax1.set_yticks([3506900,3507000,3507100,3507200,3507300,3507400,3507500])
ax1.set_yticklabels(['3506900','3507000','3507100','3507200','3507300','3507400','3507500'], size = 15)
ax1.set_xticks([727300,727400,727500,727600,727700,727800])
ax1.set_xticklabels(['727300','727400','727500','727600','727700','727800'], size = 15)
ax2 = fig.add_axes([0.78, 0.1, 0.025, 0.85])
norm = mpl.colors.Normalize(vmin=-np.max(np.abs(dobs)), vmax=np.max(np.abs(dobs)))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdBu_r, format="%.1e",ticks=[-0.99,-0.675,-0.36,-0.045,0.27,0.585,0.90]
)
cbar.set_ticklabels(
    ["-0.4","-0.2","0","0.2","0.4","0.6","0.8"],size = 17
)
cbar.set_label("mGal", rotation=270, labelpad=28, size=16.5)
vertical_lines = [(727615, 3506900, 3507200), (727540, 3507200, 3507500)]

# Add dashed horizontal lines
ax1.plot([727300, 727800], [3507350, 3507350], color='k', linestyle='--')
ax1.plot([727530, 727530], [3507500, 3507200], color='k', linestyle='--')
# Annotate points A and B
ax1.text(727300, 3507350, 'A', ha='right', va='bottom', fontsize=18)
ax1.text(727800, 3507350, 'B', ha='left', va='bottom', fontsize=18)
# Annotate points C and D
ax1.text(727530, 3507500, 'C', ha='right', va='bottom', fontsize=18)
ax1.text(727530, 3507200, 'D', ha='right', va='top', fontsize=18)
from shapely.geometry import Polygon
from shapely.geometry import Point

polygon2 = Polygon([(727645, 3507220),
   (727645, 3507420),
   (727410, 3507420),
   (727410, 3507220),
])
x, y = polygon2.exterior.xy
ax1.plot(x, y, c="black", linewidth=1.4)
maximum_anomaly = np.max(np.abs(dobs))

uncertainties = 0.01 * maximum_anomaly * np.ones(np.shape(dobs))
# Define the receivers. The data consist of vertical gravity anomaly measurements.
# The set of receivers must be defined as a list.
receiver_list = gravity.receivers.Point(receiver_locations, components="gz")

receiver_list = [receiver_list]

# Define the source field
source_field = gravity.sources.SourceField(receiver_list=receiver_list)

# Define the survey
survey = gravity.survey.Survey(source_field)
data_object = data.Data(survey, dobs=dobs, standard_deviation=uncertainties)
from discretize import TensorMesh
import matplotlib.pyplot as plt
import numpy as np
ncx = 60  # number of core mesh cells in x
ncy = 70  # number of core mesh cells in y
ncz = 40
dx = 8.33 # base cell width x
dy = 8.5715 # base cell width y
dz = 5.2
hx = dx * np.ones(ncx)
hy = dy * np.ones(ncy)
hz = dz * np.ones(ncz)
x0 = 727300
y0 = 3506900
z0 = 1580
mesh = TensorMesh([hx, hy, hz], x0=[x0, y0,z0])
# Find the indices of the active cells in forward model (ones below surface)
ind_active = active_from_xyz(mesh, xyz_topo)

# Define mapping from model to active cells
nC = int(ind_active.sum())
model_map = maps.IdentityMap(nP=nC)  # model consists of a value for each active cell

# Define and plot starting model
starting_model = np.zeros(nC)
simulation = gravity.simulation.Simulation3DIntegral(
    survey=survey, mesh=mesh, rhoMap=model_map, ind_active=ind_active
)
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# Within the data misfit, the residual between predicted and observed data are
# normalized by the data's standard deviation.
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# Within the data misfit, the residual between predicted and observed data are
# normalized by the data's standard deviation.
dmis = data_misfit.L2DataMisfit(data=data_object, simulation=simulation)
dmis.W = utils.sdiag(1 / uncertainties)

# Define the regularization (model objective function).
reg = regularization.Sparse(mesh,alpha_s = 0.5, alpha_x = 5,alpha_y = 5, alpha_z = 5,  active_cells=ind_active, mapping=model_map)
reg.norms = [0,1.0,2.0, 1.0]

# Define how the optimization problem is solved. Here we will use a projected
# Gauss-Newton approach that employs the conjugate gradient solver.
opt = optimization.ProjectedGNCG(
    maxIter= 40, lower=-1, upper = 2, maxIterLS=100, maxIterCG=20, tolCG=1e-4
)

# Here we define the inverse problem that is to be solved
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)
# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=1e0)

# Defines the directives for the IRLS regularization. This includes setting
# the cooling schedule for the trade-off parameter.
update_IRLS = directives.Update_IRLS(
    f_min_change=1e-4,
    max_irls_iterations=40,
    coolEpsFact=1.5,
    beta_tol=1e-2,
)

# Defining the fractional decrease in beta and the number of Gauss-Newton solves
# for each beta value.
beta_schedule = directives.BetaSchedule(coolingFactor=5, coolingRate=1)

# Options for outputting recovered models and predicted data for each beta.
save_iteration = directives.SaveOutputEveryIteration(save_txt=False)

# Updating the preconditionner if it is model dependent.
update_jacobi = directives.UpdatePreconditioner()

# Add sensitivity weights
sensitivity_weights = directives.UpdateSensitivityWeights(everyIter=False)

# The directives are defined as a list.
directives_list = [
    update_IRLS,
    sensitivity_weights,
    starting_beta,
    beta_schedule,
    save_iteration,
    update_jacobi,
]
# Here we combine the inverse problem and the set of directives
inv = inversion.BaseInversion(inv_prob, directives_list)

# Run inversion
recovered_model = inv.run(starting_model)


In [ ]:
# Plot Recovered Model
fig = plt.figure(figsize=(14, 5))
plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
mesh.plot_slice(
    plotting_map * recovered_model,
    normal="y",
    ax=ax1,
    ind= 55,
    pcolor_opts={"cmap": "RdYlBu"},
)
ax1.set_title(r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5,\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 28.5)
plt.xlabel("Easting (m)",size = 28,labelpad=30)
plt.ylabel("Elevation (m)",size = 28,labelpad=30)
ax2 = fig.add_axes([0.85, 0.1, 0.025, 0.8])
norm = mpl.colors.Normalize(-1, vmax=np.max(recovered_model))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r, ticks=[-1,0,1,2]
)
cbar.set_ticklabels(
    ["-1.0 ","0.0 ","1.0","2.0"],size = 23)
cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=30, size=24.5)

ax1.set_xticks([727350, 727450, 727550, 727650, 727750])
ax1.set_xticklabels(['727350', '727450', '727550', '727650', '727750'], size=21)

ax1.set_yticks([1600, 1650, 1700, 1750])
ax1.set_yticklabels(['1600', '1650', '1700', '1750'], size=21)

#ax2 = fig.add_axes([0.85, 0.1, 0.02, 0.8])
#norm = mpl.colors.Normalize(vmin=np.min(recovered_model), vmax=np.max(recovered_model))
#cbar = mpl.colorbar.ColorbarBase(
#    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.magma_r
#)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
ax1.text(0.015, 1.02, "A", transform=ax1.transAxes, size=32, ha='center')
ax1.text(0.985, 1.02, "A\'", transform=ax1.transAxes, size=32, ha='center')

ax1.text(-0.1, 1.12, "(g)", transform=ax1.transAxes, size=41, ha='center')


plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 6.5))
plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
rxLoc = survey.source_field.receiver_list[0].locations
midx = int(mesh.shape_cells[0] / 2)
yslice = midx + 5
xslice = midx - 5
ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
mesh.plot_slice(
    plotting_map * recovered_model,
    normal="z",
    ax=ax1,
    ind= 36,
    level = True,
    pcolor_opts={"cmap": "RdYlBu"},
)
ax1.set_title(r"$\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 19)
ax1.set_xlabel("Easting (m)",size = 16.5,labelpad=20)
ax1.set_ylabel("Northing (m)",size = 16.5,labelpad=20)
ax1.set_yticks([3506900,3507200,3507500])
ax1.set_yticklabels(['3506900','3507200','3507500'], size = 15)
ax1.set_xticks([727350,727550,727750])
ax1.set_xticklabels(['727350','727550','727750'], size = 15)
ax2 = fig.add_axes([0.87, 0.1, 0.045, 0.8])
norm = mpl.colors.Normalize(-1, vmax=np.max(recovered_model))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r,ticks=[-1,-0,1,2]
)
cbar.set_ticklabels(
    ["-1 ","0 ","1 ","2 "], size = 16
)
cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=17)
ax1.text(0.3, 0.22, "(A2)", transform=ax1.transAxes, size=16, ha='center')
ax1.text(0.36, 0.65, "(A1)", transform=ax1.transAxes, size=16, ha='center')
#ax1.text(0.71, 0.78, "(A3)", transform=ax1.transAxes, size=16, ha='center')
ax1.text(0.025, 1.1, "(d)", transform=ax1.transAxes, size=28, ha='center')
ax1.text(
    0.5, 1.095,  # Adjust y-coordinate to fit the plot area
    r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5$",  # Use \mathrm{...} for text in math mode
    transform=ax1.transAxes,  # Transform coordinates to axes coordinates
    size=16,
    ha='center'  # Horizontal alignment: center
)
plt.show()

In [ ]:
# Predicted data with final recovered model
dpred = inv_prob.dpred

# Observed data | Predicted data | Normalized data misfit
data_array = np.c_[dobs, dpred, (dobs - dpred) / uncertainties]

fig = plt.figure(figsize=(19, 5))
plot_title = ["Observed", "Predicted", "Normalized Misfit"]
plot_units = ["nT", "nT", ""]

ax1 = 3 * [None]
ax2 = 3 * [None]
norm = 3 * [None]
cbar = 3 * [None]
cplot = 3 * [None]
v_lim = [np.max(np.abs(dobs)), np.max(np.abs(dobs)), np.max(np.abs(data_array[:, 2]))]

for ii in range(0, 3):
    ax1[ii] = fig.add_axes([0.33 * ii + 0.03, 0.11, 0.25, 0.84])
    cplot[ii] = plot2Ddata(
        receiver_list[0].locations,
        data_array[:, ii],
        ax=ax1[ii],
        ncontour=30,
        contourOpts={"cmap": "jet"},
    )
    ax1[ii].set_title(plot_title[ii])
    ax1[ii].set_xlabel("x (m)")
    ax1[ii].set_ylabel("y (m)")

    ax2[ii] = fig.add_axes([0.33 * ii + 0.37, 0.21, 0.01, 0.84])
    norm[ii] = mpl.colors.Normalize(vmin=-v_lim[ii], vmax=v_lim[ii])
    cbar[ii] = mpl.colorbar.ColorbarBase(
        ax2[ii], norm=norm[ii], orientation="vertical", cmap=mpl.cm.jet
    )
    cbar[ii].set_label(plot_units[ii], rotation=270, labelpad=15, size=12)

#plt.show()
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib.patches import Polygon
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile
# Predicted data with final recovered model
dpred = inv_prob.dpred

fig = plt.figure(figsize=(10, 7))

ax1 = fig.add_axes([0.1, 0.1, 0.7, 0.70])
plot2Ddata(receiver_list[0].locations, ((dobs - dpred) / uncertainties), ax=ax1, ncontour=30, contourOpts={"cmap": "RdYlBu_r"})
ax1.set_title(r"$\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 19)
ax1.set_xlabel("Easting (m)", size=16.5, labelpad=20)
ax1.set_ylabel("Northing (m)", size=16.5, labelpad=20)
ax1.set_xticks([727350,727550,727750])
ax1.set_xticklabels(['727350','727550','727750'], size = 15.5)


ax1.set_yticks([3506950,3507200,3507450])
ax1.set_yticklabels(['3506950','3507200','3507450'], size = 15.5)

ax2 = fig.add_axes([0.68, 0.100, 0.025, 0.700])
norm = mpl.colors.Normalize(vmin=-v_lim[ii], vmax=v_lim[ii])
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r,ticks=[ -1,-0.75,-0.5,-0.25,0.0,0.25,0.5, 0.75, 1]
)

cbar.set_ticklabels(
    ['-1.00','-0.75','-0.50','-0.25','0.00','0.25', '0.50', '0.75', '1.00'], size=18
)
cbar.set_label(r'$\frac{d^{obs} - d^{pred}}{\sigma}$', rotation=270, labelpad=53, size=32)
ax1.text(0.025, 1.1, "(d)", transform=ax1.transAxes, size=28, ha='center')
ax1.text(
    0.5, 1.095,  # Adjust y-coordinate to fit the plot area
    r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5$",  # Use \mathrm{...} for text in math mode
    transform=ax1.transAxes,  # Transform coordinates to axes coordinates
    size=16,
    ha='center'  # Horizontal alignment: center
)
plt.show()

# Real-case magnetic susceptibility inversion

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib.patches import Polygon
import os
import tarfile
from discretize import TensorMesh
from discretize.utils import active_from_xyz
from SimPEG.potential_fields import magnetics
from SimPEG.utils import plot2Ddata, model_builder

topo_filename = "t-mag.txt"
data_filename = "Magnetic observation.txt"

# Load topography
xyz_topo = np.loadtxt(str(topo_filename))

# Load field data
dobs = np.loadtxt(str(data_filename))

# Define receiver locations and observed data
receiver_locations = dobs[:, 0:3]
dobs = dobs[:, -1]

# Plot
mpl.rcParams.update({"font.size": 12})
fig = plt.figure(figsize=(9, 7))

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.85])
plot2Ddata(receiver_locations, dobs, ax=ax1, ncontour=25,dataloc = True, contourOpts={"cmap": "RdYlBu_r"})

ax1.set_xlabel("Easting (m)", size=17.5, labelpad=25)
ax1.set_ylabel("Northing (m)", size=17.5, labelpad=25)
ax1.set_title("TMI Anomaly", size=17.5)
ax1.set_yticks([3506900, 3507000, 3507100, 3507200, 3507300, 3507400, 3507500])
ax1.set_yticklabels(['3506900', '3507000', '3507100', '3507200', '3507300', '3507400', '3507500'], size=15)
ax1.set_xticks([727300, 727400, 727500, 727600, 727700, 727800])
ax1.set_xticklabels(['727300', '727400', '727500', '727600', '727700', '727800'], size=15)

ax2 = fig.add_axes([0.78, 0.1, 0.025, 0.85])
norm = mpl.colors.Normalize(vmin=-np.max(np.abs(dobs)), vmax=np.max(np.abs(dobs)))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r, format="%.1e",
    ticks=[-17850, -11900, -5950, 0, 5950, 11900, 17850]
)
cbar.set_ticklabels(["-4000", "-2000", "0", "2000", "4000", "6000", "8000"], size=17.5)
cbar.set_label("nT", rotation=270, labelpad=25, size=19)

# Add dashed horizontal lines
ax1.plot([727300, 727800], [3507308, 3507308], color='k', linestyle='--')
ax1.plot([727300, 727800], [3507049, 3507049], color='k', linestyle='--')

# Annotate points A and B
ax1.text(727330, 3507315, 'A', ha='right', va='bottom', fontsize=20)
ax1.text(727770, 3507315, "A'", ha='left', va='bottom', fontsize=20)
# Annotate points C and D
ax1.text(727330, 3507059, 'B', ha='right', va='bottom', fontsize=20)
ax1.text(727770, 3507059, "B'", ha='left', va='bottom', fontsize=20)

# Add the new point at (727590, 3507340)
ax1.plot(727545, 3507340, 'ko', markersize=8.5)  # 'ro' means red color, circle marker
ax1.text(727535, 3507360, "Well-1", fontsize=16)
ax1.plot(727600, 3507030, 'ko', markersize=8.5)  # 'ro' means red color, circle marker
ax1.text(727570, 3506990, "Well-2", fontsize=16)

plt.show()

maximum_anomaly = np.max(np.abs(dobs))

std = 0.02 * maximum_anomaly * np.ones(len(dobs))
# Define the component(s) of the field we are inverting as a list. Here we will
# invert total magnetic intensity data.
components = ["tmi"]

# Use the observation locations and components to define the receivers. To
# simulate data, the receivers must be defined as a list.
receiver_list = magnetics.receivers.Point(receiver_locations, components=components)

receiver_list = [receiver_list]

# Define the inducing field H0 = (intensity [nT], inclination [deg], declination [deg])
inclination = 90
declination = 0
strength = 50000
inducing_field = (strength, inclination, declination)

source_field = magnetics.sources.SourceField(
    receiver_list=receiver_list, parameters=inducing_field
)

# Define the survey
survey = magnetics.survey.Survey(source_field)
data_object = data.Data(survey, dobs=dobs, standard_deviation=std)
from discretize import TensorMesh
import matplotlib.pyplot as plt
import numpy as np
ncx = 60  # number of core mesh cells in x
ncy = 70  # number of core mesh cells in y
ncz = 40
dx = 8.33 # base cell width x
dy = 8.5715 # base cell width y
dz = 5.2
hx = dx * np.ones(ncx)
hy = dy * np.ones(ncy)
hz = dz * np.ones(ncz)
x0 = 727300
y0 = 3506900
z0 = 1580
mesh = TensorMesh([hx, hy, hz], x0=[x0, y0,z0])
# Define background susceptibility model in SI. Don't make this 0!
# Otherwise the gradient for the 1st iteration is zero and the inversion will
# not converge.
background_susceptibility = 1e-4

# Find the indecies of the active cells in forward model (ones below surface)
active_cells = active_from_xyz(mesh, xyz_topo)

# Define mapping from model to active cells
nC = int(active_cells.sum())
model_map = maps.IdentityMap(nP=nC)  # model consists of a value for each cell

# Define starting model
starting_model = background_susceptibility * np.ones(nC)
# Define background susceptibility model in SI. Don't make this 0!
# Otherwise the gradient for the 1st iteration is zero and the inversion will
# not converge.
background_susceptibility = 1e-4

# Find the indecies of the active cells in forward model (ones below surface)
active_cells = active_from_xyz(mesh, xyz_topo)

# Define mapping from model to active cells
nC = int(active_cells.sum())
model_map = maps.IdentityMap(nP=nC)  # model consists of a value for each cell

# Define starting model
starting_model = background_susceptibility * np.ones(nC)

# Define the problem. Define the cells below topography and the mapping
simulation = magnetics.simulation.Simulation3DIntegral(
    survey=survey,
    mesh=mesh,
    model_type="scalar",
    chiMap=model_map,
    ind_active=active_cells,
)
# Define the data misfit. Here the data misfit is the L2 norm of the weighted
# residual between the observed data and the data predicted for a given model.
# Within the data misfit, the residual between predicted and observed data are
# normalized by the data's standard deviation.
dmis = data_misfit.L2DataMisfit(data=data_object, simulation=simulation)

# Define the regularization (model objective function)
reg = regularization.Sparse(
    mesh,alpha_s = 0.5, alpha_x = 5, alpha_y = 5, alpha_z = 5,
    active_cells=active_cells,
    mapping=model_map,
    reference_model=starting_model,
    gradient_type="total",
)

# Define sparse and blocky norms p, qx, qy, qz
reg.norms = [0,1.0,2.0, 1.0]

# Define how the optimization problem is solved. Here we will use a projected
# Gauss-Newton approach that employs the conjugate gradient solver.
opt = optimization.ProjectedGNCG(
    maxIter=40, lower=0.0, upper=1.0, maxIterLS=100, maxIterCG=20, tolCG=1e-4
)

# Here we define the inverse problem that is to be solved
inv_prob = inverse_problem.BaseInvProblem(dmis, reg, opt)
# Defining a starting value for the trade-off parameter (beta) between the data
# misfit and the regularization.
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=5)

# Options for outputting recovered models and predicted data for each beta.
save_iteration = directives.SaveOutputEveryIteration(save_txt=False)

# Defines the directives for the IRLS regularization. This includes setting
# the cooling schedule for the trade-off parameter.
update_IRLS = directives.Update_IRLS(
    f_min_change=1e-4,
    max_irls_iterations=40,
    coolEpsFact=1.5,
    beta_tol=1e-2,
)

# Updating the preconditioner if it is model dependent.
update_jacobi = directives.UpdatePreconditioner()

# Setting a stopping criteria for the inversion.
target_misfit = directives.TargetMisfit(chifact=1)

# Add sensitivity weights
sensitivity_weights = directives.UpdateSensitivityWeights(everyIter=False)

# The directives are defined as a list.
directives_list = [
    sensitivity_weights,
    starting_beta,
    save_iteration,
    update_IRLS,
    update_jacobi,
]
# Here we combine the inverse problem and the set of directives
inv = inversion.BaseInversion(inv_prob, directives_list)

# Print target misfit to compare with convergence
# print("Target misfit is " + str(target_misfit.target))

# Run the inversion
recovered_model_2 = inv.run(starting_model)

In [ ]:
# Plot Recovered Model
fig = plt.figure(figsize=(14, 5), dpi = 1100)
plotting_map = maps.InjectActiveCells(mesh, active_cells, np.nan)

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
mesh.plot_slice(
    plotting_map * recovered_model_2,
    normal="y",
    ax=ax1,
    ind=55,
    pcolor_opts={"cmap": "RdYlBu_r"},
)
ax1.set_title(r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5,\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 28.5)
plt.xlabel("Easting (m)",size = 28,labelpad=30)
plt.ylabel("Elevation (m)",size = 28,labelpad=30)
ax2 = fig.add_axes([0.85, 0.1, 0.025, 0.8])
norm = mpl.colors.Normalize(0, 1)
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r, ticks=[0,0.25,0.5,0.75,1]
)
cbar.set_ticklabels(
    ["0.00 ","0.25 ","0.50","0.75",'1.00'],size = 23)
cbar.set_label("Susceptibility (SI)", rotation=270, labelpad=30, size=24.5)
#ax2 = fig.add_axes([0.85, 0.1, 0.02, 0.8])
#norm = mpl.colors.Normalize(vmin=np.min(recovered_model), vmax=np.max(recovered_model))
#cbar = mpl.colorbar.ColorbarBase(
#    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.magma_r
#)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
ax1.set_xticks([727350, 727450, 727550, 727650, 727750])
ax1.set_xticklabels(['727350', '727450', '727550', '727650', '727750'], size=21)

ax1.set_yticks([1600, 1650, 1700, 1750])
ax1.set_yticklabels(['1600', '1650', '1700', '1750'], size=21)

#ax2 = fig.add_axes([0.85, 0.1, 0.02, 0.8])
#norm = mpl.colors.Normalize(vmin=np.min(recovered_model), vmax=np.max(recovered_model))
#cbar = mpl.colorbar.ColorbarBase(
#    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.magma_r
#)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
ax1.text(0.015, 1.02, "A", transform=ax1.transAxes, size=32, ha='center')
ax1.text(0.985, 1.02, "A\'", transform=ax1.transAxes, size=32, ha='center')

ax1.text(-0.1, 1.12, "(h)", transform=ax1.transAxes, size=41, ha='center')


plt.show()

In [ ]:
# Plot Recovered Model
fig = plt.figure(figsize=(14, 5))
plotting_map = maps.InjectActiveCells(mesh, active_cells, np.nan)

ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
mesh.plot_slice(
    plotting_map * recovered_model_2,
    normal="y",
    ax=ax1,
    ind= 17,
    pcolor_opts={"cmap": "RdYlBu_r"},
)
ax1.set_title(r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5,\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 28.5)
plt.xlabel("Easting (m)",size = 28,labelpad=30)
plt.ylabel("Elevation (m)",size = 28,labelpad=30)
ax2 = fig.add_axes([0.85, 0.1, 0.025, 0.8])
norm = mpl.colors.Normalize(0, 1)
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r, ticks=[0,0.25,0.5,0.75,1]
)
cbar.set_ticklabels(
    ["0.00 ","0.25 ","0.50","0.75",'1.00'],size = 23)
cbar.set_label("Susceptibility (SI)", rotation=270, labelpad=30, size=24.5)
#ax2 = fig.add_axes([0.85, 0.1, 0.02, 0.8])
#norm = mpl.colors.Normalize(vmin=np.min(recovered_model), vmax=np.max(recovered_model))
#cbar = mpl.colorbar.ColorbarBase(
#    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.magma_r
#)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
ax1.set_xticks([727350, 727450, 727550, 727650, 727750])
ax1.set_xticklabels(['727350', '727450', '727550', '727650', '727750'], size=21)

ax1.set_yticks([1600, 1650, 1700, 1750])
ax1.set_yticklabels(['1600', '1650', '1700', '1750'], size=21)

#ax2 = fig.add_axes([0.85, 0.1, 0.02, 0.8])
#norm = mpl.colors.Normalize(vmin=np.min(recovered_model), vmax=np.max(recovered_model))
#cbar = mpl.colorbar.ColorbarBase(
#    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.magma_r
#)
#cbar.set_label("Density (gr/cm³)", rotation=270, labelpad=25, size=18)
ax1.text(0.015, 1.02, "B", transform=ax1.transAxes, size=32, ha='center')
ax1.text(0.985, 1.02, "B\'", transform=ax1.transAxes, size=32, ha='center')

ax1.text(-0.1, 1.12, "(h)", transform=ax1.transAxes, size=41, ha='center')


plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 6.5))
plotting_map = maps.InjectActiveCells(mesh, ind_active, np.nan)
rxLoc = survey.source_field.receiver_list[0].locations
midx = int(mesh.shape_cells[0] / 2)
yslice = midx + 5
xslice = midx - 5
ax1 = fig.add_axes([0.1, 0.1, 0.73, 0.8])
mesh.plot_slice(
    plotting_map * recovered_model_2,
    normal="z",
    ax=ax1,
    ind= 36,
    level = True,
    pcolor_opts={"cmap": "RdYlBu_r"},
)
ax1.set_title(r"$\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 19)
ax1.set_xlabel("Easting (m)",size = 16.5,labelpad=20)
ax1.set_ylabel("Northing (m)",size = 16.5,labelpad=20)
ax1.set_yticks([3506900,3507200,3507500])
ax1.set_yticklabels(['3506900','3507200','3507500'], size = 15)
ax1.set_xticks([727350,727550,727750])
ax1.set_xticklabels(['727350','727550','727750'], size = 15)
ax2 = fig.add_axes([0.87, 0.1, 0.045, 0.8])
norm = mpl.colors.Normalize(0, vmax=np.max(recovered_model_2))
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r,ticks=[0,0.25,0.5,0.75,1]
)
cbar.set_ticklabels(
    ["0.00","0.25 ","0.50 ","0.75 ", '1.00'], size = 16
)
cbar.set_label("Susceptibility (SI)", rotation=270, labelpad=25, size=17)
ax1.text(0.5, 0.22, "(A3)", transform=ax1.transAxes, size=16, ha='center', color =  'white')
ax1.text(0.335, 0.65, "(A1)", transform=ax1.transAxes, size=16, ha='center',  color ='white')
ax1.text(0.445, 0.48, "(A2)", transform=ax1.transAxes, size=16, ha='center',  color ='white')
ax1.text(0.025, 1.1, "(d)", transform=ax1.transAxes, size=28, ha='center')
ax1.text(
    0.5, 1.095,  # Adjust y-coordinate to fit the plot area
    r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5$",  # Use \mathrm{...} for text in math mode
    transform=ax1.transAxes,  # Transform coordinates to axes coordinates
    size=16,
    ha='center'  # Horizontal alignment: center
)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib.patches import Polygon
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile
# Predicted data with final recovered model
dpred = inv_prob.dpred

fig = plt.figure(figsize=(10, 7))

ax1 = fig.add_axes([0.1, 0.1, 0.7, 0.70])
plot2Ddata(receiver_list[0].locations, ((dobs - dpred) / std), ax=ax1, ncontour=30, contourOpts={"cmap": "RdYlBu_r"}, clim = (-v_lim[ii],v_lim[ii]))
ax1.set_title(r"$\phi_m^p = \phi_S^0 + \phi_X^1 + \phi_Y^2 + \phi_Z^1$", size = 19)
ax1.set_xlabel("Easting (m)", size=16.5, labelpad=20)
ax1.set_ylabel("Northing (m)", size=16.5, labelpad=20)
ax1.set_xticks([727350,727550,727750])
ax1.set_xticklabels(['727350','727550','727750'], size = 15.5)


ax1.set_yticks([3506950,3507200,3507450])
ax1.set_yticklabels(['3506950','3507200','3507450'], size = 15.5)

ax2 = fig.add_axes([0.68, 0.100, 0.025, 0.700])
norm = mpl.colors.Normalize(vmin=-v_lim[ii], vmax=v_lim[ii])
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r,ticks=[ -10,-7.5,-5,-2.5,0.0,2.5,5, 7.5, 10]
)

cbar.set_ticklabels(
    ['-10.0','-7.5','-5.0','-2.5','0.0','2.5', '5.0', '7.5', '10.0'], size=18
)
cbar.set_label(r'$\frac{d^{obs} - d^{pred}}{\sigma}$', rotation=270, labelpad=53, size=32)
ax1.text(0.025, 1.1, "(d)", transform=ax1.transAxes, size=28, ha='center')
ax1.text(
    0.5, 1.095,  # Adjust y-coordinate to fit the plot area
    r"$\alpha_s = 0.5,\alpha_{x,y,z} = 5$",  # Use \mathrm{...} for text in math mode
    transform=ax1.transAxes,  # Transform coordinates to axes coordinates
    size=16,
    ha='center'  # Horizontal alignment: center
)
plt.show()